# Collecting dynamic information with test cases

In [2]:
import pytest

In [3]:
from types import FrameType, TracebackType
from typing import Any, Optional, Callable

import io

output = io.StringIO()

def traceit(frame: FrameType, event: str, arg: Any) -> Optional[Callable]:
#     print(event, frame.f_lineno, frame.f_code.co_name, frame.f_locals)
    if frame.f_code.co_filename.endswith("sessions.py"):
        #     print(*args, file=output, **kwargs)
        print(frame.f_lineno, event, frame.f_code.co_name, frame.f_code.co_qualname, "\n", file=output)
    return traceit

In [4]:
import sys

sys.settrace(traceit)
pytest.main(["-qqqq", "./cli"])
sys.settrace(None)

contents = output.getvalue()
output.close()

.................

Downloading to /var/folders/wz/1s6cf6yx1w768h9dr46pvs8h0000gn/T/httpie_stderrlp_tek2z

Done. 30 bytes in 00:0.06904 (434.51733269167937 bytes/s)

F.....

http: error: ConnectionError: ('Connection aborted.', HTTPException('got more than 1 headers')) while doing a GET request to URL: http://127.0.0.1:56708/get

...

usage:
    http [METHOD] URL [REQUEST_ITEM ...]

error:
    Only terminal output can be colorized on Windows.

for more information:
    run 'http --help' or visit https://httpie.io/docs/cli

F................. [ 98%]
....................                                                     [100%]
=================================== FAILURES ===================================
___________________ TestDownloads.test_actual_download[http] ___________________

self = <tests.test_downloads.TestDownloads object at 0x11855e9d0>
httpbin_both = <pytest_httpbin.serve.Server object at 0x118afee90>
httpbin = <pytest_httpbin.serve.Server object at 0x118afee90>

    def test_actual_download(self, httpbin_both, httpbin):
        robots_txt = '/robots.txt'
        body = urlopen(httpbin + robots_txt).read().decode()
        env = MockEnvironment(stdin_isatty=True, stdout_isatty=False, show_displays=True)
        r = http('--download', httpbin_both.url + robots_txt, env=env)
>       assert 'Downloading' in r.stderr
E       AssertionError: assert 'Downloading' in ''
E        +  where '' = 'User-agent: *\nDisallow: /deny\n'.stderr

cli/tests/test_downloads.py:130: AssertionError
-------------

________________________ test_raw_POST_redirected_stdin ________________________

httpbin = <pytest_httpbin.serve.Server object at 0x118afee90>

    def test_raw_POST_redirected_stdin(httpbin):
        r = http(
            '--raw',
            'foo bar',
            'POST',
            httpbin + '/post',
            tolerate_error_exit_status=True,
            env=MockEnvironment(
                stdin='some=value',
                stdin_isatty=False,
            ),
        )
        assert r.exit_status == ExitStatus.ERROR
>       assert 'cannot be mixed' in r.stderr
E       AssertionError: assert 'cannot be mixed' in '\n'
E        +  where '\n' = ''.stderr

cli/tests/test_httpie.py:172: AssertionError
----------------------------- Captured stderr call -----------------------------

______________ test_httpie_sessions_upgrade_on_non_existent_file _______________

tmp_path = PosixPath('/private/var/folders/wz/1s6cf6yx1w768h9dr46pvs8h0000gn/T/pytest-of-darkrsw/pytest-16/test_httpie_ses

    @pytest.mark.parametrize('headers, expect_warning', [
        # Old-style header format
        (
            {},
            False
        ),
        (
            {'Foo': 'bar'},
            True
        ),
        # New style header format
        (
            [],
            False
        ),
        (
            [{'name': 'Foo', 'value': 'Bar'}],
            False
        ),
    ])
    def test_headers_old_layout_warning(basic_session, mock_env, headers, expect_warning):
        with open_raw_session(basic_session) as raw_session:
            raw_session['headers'] = headers
    
        with open_session(basic_session, mock_env, read_only=True):
            warning = b'Outdated layout detected'
            stderr = read_stderr(mock_env)
    
            if expect_warning:
>               assert warning in stderr
E               AssertionError: assert b'Outdated layout detected' in b''

cli/tests/test_sessions.py:739: AssertionError
---------------------------- Captured stder

In [1]:
print(contents)

NameError: name 'contents' is not defined